In [26]:
import json

data_file = open("./training_data_set.json")
raw_data = json.load(data_file)
data_file.close()

sites = raw_data.get("sitesAndTags", [])
print("Quantidade total de sites:", len(sites))

Quantidade total de sites: 200


In [27]:
from sklearn.model_selection import train_test_split
import requests
from bs4 import BeautifulSoup

# Function to remove tags
def remove_tags(html):
    soup = BeautifulSoup(html, "html.parser")
  
    for data in soup(['style', 'script']):
        data.decompose()
  
    return ' '.join(soup.stripped_strings)

sites_contents = []
y_sites = []

for [url, usefulness] in sites:
    r = requests.get(url)
    filtered_content = remove_tags(r.content)
    sites_contents.append(filtered_content)
    y_sites.append(usefulness)
    print(url)

https://www.bigboygames.com.br/jogo-fallen-legion-revenants-vanguard-edition-nintendo-switch
https://www.bigboygames.com.br/world-of-warriors-ps4-4955-p993556
https://www.bigboygames.com.br/jogo-monster-hunter-world-seminovo-ps4-11807-br-p1006227
https://www.bigboygames.com.br/jogo-back-4-blood-xbox-one
https://www.bigboygames.com.br/jogo-far-cry-6-xbox-one-p1006787
https://www.bigboygames.com.br/controle-baseus-sem-fio-transparente-seminovo-nintendo-switch
https://www.bigboygames.com.br/cartao-xbox-live-brasil-r200-5598-p995690
https://www.bigboygames.com.br/case-zelda-botw-seminovo-nintendo-switch-lite-15816
https://www.bigboygames.com.br/console-nintendo-switch-lite-amarelo-seminovo-16101
https://www.bigboygames.com.br/case-protetora-nintendo-swtich-lite-cinza-amarelo-14940
https://www.ibyte.com.br/jogo-hades-xbox/p
https://www.ibyte.com.br/marvel-s-spider-man-miles-morales-ps5/p
https://www.ibyte.com.br/game-fifa-18-xbox-one/p
https://www.shockgames.com.br/produto/horizon-zero-dawn

In [28]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sites_contents, y_sites, random_state = 1337)

Optimizing Random Forest

In [30]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np

rf_text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier()),
])

max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

rf_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'vect__min_df': (0.001, 0.005, 0.01, 0.05, 0.1),
    'vect__max_features': (500, 1000, 1500, 2000, 2500, 5000, 10000),
    'tfidf__use_idf': (True, False),
    'clf__n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
    'clf__max_depth': max_depth,
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4],
    'clf__bootstrap': [True, False]
}

randomized_gs_rf_clf = RandomizedSearchCV(rf_text_clf, rf_parameters, cv=5, n_jobs=-1)

In [31]:
randomized_gs_rf_clf.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                             ('tfidf', TfidfTransformer()),
                                             ('clf',
                                              RandomForestClassifier())]),
                   n_jobs=-1,
                   param_distributions={'clf__bootstrap': [True, False],
                                        'clf__max_depth': [10, 20, 30, 40, 50,
                                                           60, 70, 80, 90, 100,
                                                           110, None],
                                        'clf__min_samples_leaf': [1, 2, 4],
                                        'clf__min_samples_split': [2, 5, 10],
                                        'clf__n_estimators': [200, 400, 600,
                                                              800, 1000, 1200,
                                                              1400, 1600, 1800,
                                                              2000],
                                        'tfidf__use_idf': (True, False),
                                        'vect__max_features': (500, 1000, 1500,
                                                               2000, 2500, 5000,
                                                               10000),
                                        'vect__min_df': (0.001, 0.005, 0.01,
                                                         0.05, 0.1),
                                        'vect__ngram_range': [(1, 1), (1, 2)]})

In [32]:
randomized_gs_rf_clf.best_score_

0.8533333333333333

In [34]:
for param_name in sorted(rf_parameters.keys()):
    print("%s: %r" % (param_name, randomized_gs_rf_clf.best_params_[param_name]))

print("Stats for optimized random forest")

pred = randomized_gs_rf_clf.predict(X_test)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, pred, labels=[True, False])
print("Precision:", precision)
print("recall:", recall)
print("f1:", f1)
print("===============================================================")

clf__bootstrap: False
clf__max_depth: 40
clf__min_samples_leaf: 1
clf__min_samples_split: 10
clf__n_estimators: 200
tfidf__use_idf: False
vect__max_features: 10000
vect__min_df: 0.005
vect__ngram_range: (1, 2)
Stats for optimized random forest
Precision: [0.95652174 0.81481481]
recall: [0.81481481 0.95652174]
f1: [0.88 0.88]


Optimizing SVM

In [35]:
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=1337,
                          max_iter=5, tol=None)),
])

parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'vect__min_df': (0.001, 0.005, 0.01, 0.05, 0.1),
    'vect__max_features': range(1000, 10000, 1000),
    'tfidf__use_idf': (True, False),
    'clf__alpha': (1e-2, 1e-3),
}

gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)

In [37]:
gs_clf.fit(X_train, y_train)

In [ ]:
gs_clf.best_score_

In [ ]:
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

print("Stats for optimized svm")

pred = gs_clf.predict(X_test)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, pred, labels=[True, False])
print("Precision:", precision)
print("recall:", recall)
print("f1:", f1)
print("===============================================================")